In [1]:
!pip install pandas langchain faiss-cpu sentence-transformers transformers langchain-community

In [2]:
import pandas as pd

# Load the Kaggle loan dataset
df = pd.read_csv("Training Dataset.csv")

# Drop missing values
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Preview the dataset
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [3]:
documents = []

for idx, row in df.iterrows():
    text = (
        f"Applicant ID: {row['Loan_ID']}. "
        f"Gender: {row['Gender']}, Married: {row['Married']}, Dependents: {row['Dependents']}, "
        f"Education: {row['Education']}, Self_Employed: {row['Self_Employed']}, "
        f"ApplicantIncome: {row['ApplicantIncome']}, CoapplicantIncome: {row['CoapplicantIncome']}, "
        f"LoanAmount: {row['LoanAmount']}, Loan_Amount_Term: {row['Loan_Amount_Term']}, "
        f"Credit_History: {row['Credit_History']}, Property_Area: {row['Property_Area']}, "
        f"Loan_Status: {row['Loan_Status']}."
    )
    documents.append(text)

# Check one sample
print("Sample document:\n", documents[0])


Sample document:
 Applicant ID: LP001003. Gender: Male, Married: Yes, Dependents: 1, Education: Graduate, Self_Employed: No, ApplicantIncome: 4583, CoapplicantIncome: 1508.0, LoanAmount: 128.0, Loan_Amount_Term: 360.0, Credit_History: 1.0, Property_Area: Rural, Loan_Status: N.


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents)
embeddings = np.array(embeddings).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [5]:
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document

# Wrap documents for FAISS
documents_dict = {str(i): Document(page_content=documents[i]) for i in range(len(documents))}
docstore = InMemoryDocstore(documents_dict)
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

# Rebuild FAISS correctly
vectorstore = FAISS(
    embedding_function=doc_embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
)


NameError: name 'FAISS' is not defined

In [ ]:
query = "What is the average loan amount for approved applicants?"
result = qa_chain.invoke({"query": query})
print(result["result"])


In [ ]:
import pickle
with open("rag_index.pkl", "wb") as f:
    pickle.dump((index, documents), f)


In [ ]:
approved = df[df['Loan_Status'] == 'Y']
avg_loan = approved['LoanAmount'].mean()
print("Average Loan Amount for Approved Applicants:", round(avg_loan, 2))
